In [46]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [47]:
import tensorflow as tf
import numpy as np
from tensorflow.python.client import device_lib
import tensorflow.contrib.immediate as immediate

In [48]:
device_names = [d.name for d in device_lib.list_local_devices()]
print device_names

[u'/cpu:0', u'/gpu:0']


In [42]:
import os
print os.environ['CUDA_VISIBLE_DEVICES']

1


In [4]:
ls ../data

t10k-images-idx3-ubyte.gz  train-images-idx3-ubyte.gz
t10k-labels-idx1-ubyte.gz  train-labels-idx1-ubyte.gz


## Load MNIST data

In [5]:
train_data_filename="../data/train-images-idx3-ubyte.gz"
train_labels_filename="../data/train-labels-idx1-ubyte.gz"

In [30]:
import gzip
def extract_data(filename, num_images):
  with gzip.open(filename) as bytestream:
    bytestream.read(16)
    buf = bytestream.read(28 * 28 * num_images)
    data = np.frombuffer(buf, dtype=np.uint8)
    data = data.reshape(num_images, 28, 28, 1)
    return data

def extract_labels(filename, num_images):
  """Extract the labels into a vector of int64 label IDs."""
  with gzip.open(filename) as bytestream:
    bytestream.read(8)
    buf = bytestream.read(1 * num_images)
    labels = np.frombuffer(buf, dtype=np.uint8)
  return labels

In [31]:
import tensorflow.models.image.mnist.convolutional as convolutional
train_data = extract_data(train_data_filename, 10000)
train_labels = extract_labels(train_labels_filename, 10000)

In [32]:
len(train_data)

10000

In [33]:
from PIL import Image

In [34]:
Image.fromarray(train_data[0,:,:,0])

In [14]:
train_data[0].shape

(28, 28, 1)

In [36]:
train_data[0,:,16:30,0]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0],
       [253, 253, 225, 172, 253, 242, 195,  64,   0,   0,   0,   0],
       [253, 251,  93,  82,  82,  56,  39,   0,   0,   0,   0,   0],
       [247, 241,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [ 43, 154,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [108,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [253, 119,  25,   0,   0,  

In [39]:
train_labels[0:6]

array([5, 0, 4, 1, 9, 2], dtype=uint8)

In [44]:
train_data[0].shape

(28, 28, 1)

In [ ]:
    0    0    0    0    0    0    0    0    0    0   30   36   94  154  170


In [51]:
train_data[0,6,0:20,0]

array([  0,   0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 170,
       253, 253, 253, 253, 253, 225, 172], dtype=uint8)

## Convert MNIST csv data to numpy

In [23]:
data = np.loadtxt("mnist.t7/test_32x32.t7.csv.data", dtype=np.uint8).reshape(10000, 32, 32)

In [27]:
np.save("mnist.t7/test_32x32.npy", data)

In [30]:
data = np.loadtxt("mnist.t7/train_32x32.t7.csv.data", dtype=np.uint8).reshape(60000, 32, 32)
np.save("mnist.t7/train_32x32.npy", data)

In [31]:
data = np.loadtxt("mnist.t7/train_32x32.t7.csv.labels", dtype=np.uint8)
np.save("mnist.t7/train_labels.npy", data)

In [32]:
data = np.loadtxt("mnist.t7/test_32x32.t7.csv.labels", dtype=np.uint8)
np.save("mnist.t7/test_labels.npy", data)

In [33]:
train_data = np.load("mnist.t7/train_32x32.npy")

In [42]:
train_data[0,8,:]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  30,  36,  94,
       154, 170, 253, 253, 253, 253, 253, 225, 172, 253, 242, 195,  64,
         0,   0,   0,   0,   0,   0], dtype=uint8)

In [43]:
train_labels=np.load("mnist.t7/train_labels.npy")

In [45]:
train_labels[0:10]

array([ 6,  1,  5,  2, 10,  3,  2,  4,  2,  5], dtype=uint8)

In [ ]:
tf.Fif

# Power iteration

In [2]:
import tensorflow as tf
from tensorflow.contrib import immediate
env = immediate.Env(tf)
ti = env.tf

In [7]:
def rank1_approximation(a):
    a = a/ti.reduce_sum(a)
    u = ti.reduce_sum(a, reduction_indices=[0], keep_dims=True)
    return ti.matmul(ti.transpose(u), u)

n = 2048
a = ti.random_uniform((n, n))
a = a+ti.transpose(a)
b = a
for i in range(100):
    print("Iterating")
    b_old = b
    b = ti.matmul(b, a)
    b = b/ti.reduce_sum(b)
    
    normdiff = ti.reduce_sum(ti.abs(b-b_old))
    if normdiff < 1e-10:
        break
        
ti.reduce_sum(rank1_approximation(b)-b)

Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating


ITensor(1.53209e-07, dtype=float32)

In [17]:
import time
def rank1_approximation(a):
    a = a/ti.reduce_sum(a)
    u = ti.reduce_sum(a, reduction_indices=[0], keep_dims=True)
    return ti.matmul(ti.transpose(u), u)

n = 2048
a = ti.random_uniform((n, n))
a = a+ti.transpose(a)
b = a
start_time = time.time()
for i in range(100):
    if i%10==0:
        print time.time()-start_time
    start_time = time.time()
    b_old = b
    with tf.device("gpu:0"):
        b = ti.matmul(b, a)
        b = b/ti.reduce_sum(b)
    
    normdiff = ti.reduce_sum(ti.abs(b-b_old))
    if normdiff < 1e-10:
        break
        
ti.reduce_sum(rank1_approximation(b)-b)

0.00040602684021
0.0288240909576
0.0283560752869
0.0279960632324
0.0270628929138
0.0279879570007
0.0290751457214
0.0284898281097
0.0283360481262
0.0286450386047


ITensor(4.9547e-07, dtype=float32)

# Matmul benchmark

## Native TensorFlow

In [6]:
import time
import tensorflow as tf

### Native copy-free

In [51]:
tf.reset_default_graph()
n = 1024*4
n_ops = n**3+n*n*(n-1)  # n**3 muls, n**2*(n-1) adds
a = tf.ones((n, n), dtype=tf.float32)/n
ab = tf.matmul(a, a)
ab_op = tf.group(ab)

sess = tf.Session()
speeds = []
for i in range(10):
    start_time = time.time()
    sess.run(ab_op)
    end_time = time.time()
    elapsed_time = end_time - start_time
    speed = n_ops/elapsed_time/10**9
    speeds.append(speed)
print("max speed: %s T ops/sec" %(max(speeds)))

max speed: 2630.64627878 T ops/sec


### Native with Variables

In [50]:
import tensorflow as tf
import time

tf.reset_default_graph()
n = 1024*2
n_ops = n**3+n*n*(n-1)  # n**3 muls, n**2*(n-1) adds
a = tf.ones((n, n), dtype=tf.float32)/n
a_fixed = tf.Variable(a)
b = tf.Variable(a)
init_op = tf.initialize_all_variables()
ab = tf.matmul(a, b)
update_op = b.assign(ab)

config = tf.ConfigProto(log_device_placement=False)
sess = tf.Session(config=config)

sess.run(init_op)
speeds = []
start_time = time.time()
for i in range(10):
    sess.run(update_op)
    elapsed_time = time.time() - start_time
    speed = n_ops/elapsed_time/10**9
    start_time = time.time()
    speeds.append(speed)
    
print("max speed: %s T ops/sec" %(max(speeds)))

max speed: 1428.20328407 T ops/sec


## Immediate version

In [59]:
import tensorflow as tf
from tensorflow.contrib import immediate
import time

env = immediate.Env(tf)
ti = env.tf

n = 2024
n_ops = n**3+n*n*(n-1)  # n**3 muls, n**2*(n-1) adds
a = ti.ones((n, n), dtype=tf.float32)/n
b = ti.ones((n, n), dtype=tf.float32)/n
speeds = []
for i in range(10):
    start_time = time.time()
    b = ti.matmul(a, b)
    elapsed_time = time.time() - start_time
    speed = n_ops/elapsed_time/10**9
    speeds.append(speed)
    
print("max speed: %s T ops/sec" %(max(speeds)))

max speed: 2276.3841887 T ops/sec


In [55]:
ti.reduce_sum(b)

ITensor(2023.87, dtype=float32)

### Immediate on CPU

In [60]:
import tensorflow as tf
from tensorflow.contrib import immediate
import time

env = immediate.Env(tf)
ti = env.tf

n = 2024
n_ops = n**3+n*n*(n-1)  # n**3 muls, n**2*(n-1) adds
speeds = []
with env.g.device("cpu:0"):
    a = ti.ones((n, n), dtype=tf.float32)/n
    b = ti.ones((n, n), dtype=tf.float32)/n
    for i in range(10):
        start_time = time.time()
        b = ti.matmul(a, b)
        elapsed_time = time.time() - start_time
        speed = n_ops/elapsed_time/10**9
        speeds.append(speed)
    
print("max speed: %s T ops/sec" %(max(speeds)))

max speed: 147.200035166 T ops/sec
